### Ray Tune
For this tutorial we will rely `ray.tune` which can be employed to identify the best set of hyper parameters

In [33]:
from ray import tune

For `tune` to work, it is required to have at least two things:
- Objective function (model training function)
- Search space bounds and strategies

In [27]:
# define an objective function that returns a score metric
def objective(config):
    score = config["a"] ** 2 + config["b"]
    return {"function_score": score}

In [21]:
# define the approach and bounds to search for hyper parameters
config = {
    # Do a grid search over these values. Every value will be sampled
    # ``num_samples`` times (``num_samples`` is the parameter you pass to ``tune.TuneConfig``,
    # which is taken in by ``Tuner``)
    "a": tune.grid_search([0.001, 0.01, 0.1, 1.0]),
    
    # Sample an option uniformly from the specified choices
    "b": tune.choice([1, 2, 3]),
}

Examples for available search strategies

In [ ]:
# config = {
#     # Sample a float uniformly between -5.0 and -1.0
#     "uniform": tune.uniform(-5, -1),

#     # Sample a float uniformly between 3.2 and 5.4,
#     # rounding to multiples of 0.2
#     "quniform": tune.quniform(3.2, 5.4, 0.2),

#     # Sample a float uniformly between 0.0001 and 0.01, while
#     # sampling in log space
#     "loguniform": tune.loguniform(1e-4, 1e-2),

#     # Sample a float uniformly between 0.0001 and 0.1, while
#     # sampling in log space and rounding to multiples of 0.00005
#     "qloguniform": tune.qloguniform(1e-4, 1e-1, 5e-5),

#     # Sample a random float from a normal distribution with
#     # mean=10 and sd=2
#     "randn": tune.randn(10, 2),

#     # Sample a random float from a normal distribution with
#     # mean=10 and sd=2, rounding to multiples of 0.2
#     "qrandn": tune.qrandn(10, 2, 0.2),

#     # Sample a integer uniformly between -9 (inclusive) and 15 (exclusive)
#     "randint": tune.randint(-9, 15),

#     # Sample a random uniformly between -21 (inclusive) and 12 (inclusive (!))
#     # rounding to multiples of 3 (includes 12)
#     # if q is 1, then randint is called instead with the upper bound exclusive
#     "qrandint": tune.qrandint(-21, 12, 3),

#     # Sample a integer uniformly between 1 (inclusive) and 10 (exclusive),
#     # while sampling in log space
#     "lograndint": tune.lograndint(1, 10),

#     # Sample a integer uniformly between 1 (inclusive) and 10 (inclusive (!)),
#     # while sampling in log space and rounding to multiples of 2
#     # if q is 1, then lograndint is called instead with the upper bound exclusive
#     "qlograndint": tune.qlograndint(1, 10, 2),

#     # Sample from a random function, in this case one that
#     # depends on another value from the search space
#     "func": tune.sample_from(lambda spec: spec.config.uniform * 0.01),
# }

Basic example run with default settings (Note that the results are displayed in a dark font that does not adapt to vs code theme)

In [34]:
if ray.is_initialized:
    ray.shutdown()
ray.init()

tuner = tune.Tuner(objective, param_space=config)
results = tuner.fit()

print("Best config is:", results.get_best_result(metric="function_score", mode="max").config)

2023-05-26 15:38:09,686	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Trial name,date,done,experiment_tag,function_score,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
objective_d8d14_00000,2023-05-26_15-38-12,True,"0_a=0.0010,b=2",2,Sansfil-Eduroam-Etudiants-25-238.polymtl.ca,1,127.0.0.1,94858,6.7234e-05,6.7234e-05,6.7234e-05,1685129892,1,d8d14_00000


2023-05-26 15:38:13,614	INFO tune.py:945 -- Total run time: 2.99 seconds (2.96 seconds for the tuning loop).


Best config is: {'a': 0.01, 'b': 3}


Let's change some of the default settings,

In [38]:
if ray.is_initialized:
    ray.shutdown()
ray.init()

tuner = tune.Tuner(objective, 
tune_config=tune.TuneConfig(
        metric="function_score",
        mode="max",
        num_samples=50,
    ),
    param_space=config)
results = tuner.fit()

print("Best config is:", results.get_best_result(metric="function_score").config)

2023-05-26 15:42:53,437	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Trial name,date,done,experiment_tag,function_score,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
objective_81e1c_00000,2023-05-26_15-42-55,True,"0_a=0.0010,b=3",3,Sansfil-Eduroam-Etudiants-25-238.polymtl.ca,1,127.0.0.1,95255,9.5129e-05,9.5129e-05,9.5129e-05,1685130175,1,81e1c_00000
objective_81e1c_00008,2023-05-26_15-42-55,True,"8_a=0.0010,b=1",1,Sansfil-Eduroam-Etudiants-25-238.polymtl.ca,1,127.0.0.1,95255,9.91821e-05,9.91821e-05,9.91821e-05,1685130175,1,81e1c_00008


Best config is: {'a': 1.0, 'b': 3}


In [39]:
ray.shutdown()